## Importação e Preparação da base

In [1]:
# %%capture
# !pip install scikit-optimize
# !pip install openml
# !pip install optuna
# !pip install neupy
# !pip install --upgrade neupy
# !pip install --upgrade theano
# !pip install imbalanced-learn

In [3]:
# Verificação de pacotes
#%%capture
import sys
import subprocess
import pkg_resources

# Função para instalar pacotes, se necessário
def install_package(package):
    try:
        # Verificar se o pacote já está instalado
        dist = pkg_resources.get_distribution(package)
        print(f"'{package}' já está instalado: versão {dist.version}")
    except pkg_resources.DistributionNotFound:
        # Instalar o pacote, se não estiver instalado
        print(f"Instalando '{package}'...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Lista de pacotes necessários
packages = [
    "scikit-optimize",
    "openml",
    "optuna",
    "neupy",
    "theano",
    "imbalanced-learn",
    "openpyxl",
    "scikit-posthocs"
]

# Verificar se está no Google Colab
try:
    import google.colab
    is_colab = True
    print("Detectado ambiente Google Colab.")
except ImportError:
    is_colab = False
    print("Ambiente local detectado.")

# Instalar pacotes
for package in packages:
    if is_colab or package not in ["neupy", "theano"]:  # 'neupy' e 'theano' apenas em ambiente local
        install_package(package)

# Atualizar pacotes específicos
if not is_colab:
    print("Atualizando 'neupy' e 'theano' no ambiente local...")
    # Install the latest version of NeuPy#+
    !pip install --upgrade neupy
    # Install the required package
    !pip install --upgrade theano

Ambiente local detectado.
'scikit-optimize' já está instalado: versão 0.10.2
'openml' já está instalado: versão 0.15.0
'optuna' já está instalado: versão 4.1.0
'imbalanced-learn' já está instalado: versão 0.12.4
'openpyxl' já está instalado: versão 3.1.5
Instalando 'scikit-posthocs'...
Atualizando 'neupy' e 'theano' no ambiente local...


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for neupy from https://files.pythonhosted.org/packages/21/be/19082cbe9a6c76dd909255341587f0b487cd3e9d32d44debda013d2accd1/neupy-0.8.2-py2.py3-none-any.whl.metadata
  Using cached neupy-0.8.2-py2.py3-none-any.whl.metadata (1.1 kB)
INFO: pip is looking at multiple versions of neupy to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for neupy from https://files.pythonhosted.org/packages/c9/da/fec21f526864b850dea52a9d09e633d303c78588aca3af00a314e41960a4/neupy-0.8.1-py2.py3-none-any.whl.metadata
  Using cached neupy-0.8.1-py2.py3-none-any.whl.metadata (1.1 kB)
  Obtaining dependency information for neupy from https://files.pythonhosted.org/packages/9c/a2/4d8dc5d686adcdd3ed2c98c85ccf69370ef0675574b0f8dbb9b91abc978c/neupy-0.8.0-py2.py3-none-any.whl.metadata
  Using cached neupy-0.8.0-py2.py3-none-any.whl.metadata (1.1 kB)
  Obtaining dependency information for neupy from https://file

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
neupy 0.6.5 requires Theano==1.0.0, but you have theano 1.0.5 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%%capture
import pandas as pd
from scipy.io import arff
import openml
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Definir o ID do dataset no OpenML
dataset_id = 722

# Carregar o dataset diretamente do OpenML
dataset = openml.datasets.get_dataset(dataset_id)
data = dataset.get_data()[0]  # Pega o DataFrame completo

# Remove colunas onde todos os valores são zero: [f10-f12] e [f34-f48]
data = data.loc[:, (data != 0).any(axis=0)]

In [5]:
data.columns

Index(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f13', 'f14',
       'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24',
       'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33',
       'binaryClass'],
      dtype='object')

## Utilizando ADASYN

In [6]:
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import ADASYN
import pandas as pd

# Definir a variável alvo
target_column = 'binaryClass'
X = data.drop(columns=[target_column])
y = data[target_column]

# Supondo que y tenha as classes 'N' e 'P' como strings
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Converte as classes para números (0 e 1)


# Dividir os dados em treino e teste com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Definir o StratifiedKFold com 10 folds para garantir a estratificação nas validações cruzadas
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Identificar colunas numéricas e ajustar o tipo de dado para float64
numeric_features = X_train.select_dtypes(include=['uint8']).columns
X_train[numeric_features] = X_train[numeric_features].astype('float64')
X_test[numeric_features] = X_test[numeric_features].astype('float64')

# Substituir valores ausentes pela média em cada coluna numérica
X_train[numeric_features] = X_train[numeric_features].fillna(X_train[numeric_features].mean())
X_test[numeric_features] = X_test[numeric_features].fillna(X_train[numeric_features].mean())

# Aplicar transformação (padronização) nas colunas numéricas
scaler = StandardScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

# Aplicar ADASYN para balancear as classes no conjunto de treino
adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)

# Agora X_train_resampled e y_train_resampled estão prontos para uso em modelos com 10-fold cross-validation


## Início experimento - Variáveis Básicas


In [7]:
# Realizando experimento já considerando X_train_resampled e y_train_resampled
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
# from neupy import algorithms

# Configurar K-Fold com estratificação
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Definição básica dos classificadores
knn = KNeighborsClassifier()
lvq = MLPClassifier(solver='sgd', learning_rate='constant', learning_rate_init=0.1, max_iter=100)
tree = DecisionTreeClassifier()
svm = SVC(probability=True)
rf = RandomForestClassifier()
mlp = MLPClassifier(max_iter=100, hidden_layer_sizes=(50,))
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
lgbm = LGBMClassifier(n_estimators=50, max_depth=3)

# Comitê de Redes Neurais Artificiais
ann_ensemble = VotingClassifier(
    estimators=[
        ('mlp_relu', MLPClassifier(activation='relu', hidden_layer_sizes=(50, 20), max_iter=100)),
        ('mlp_tanh', MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 20), max_iter=100)),
        ('mlp_logistic', MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 20), max_iter=100))
    ],
    voting='soft',
    n_jobs=-1  # Paralelização
)

# Comitê Heterogêneo (Stacking)
stacking = StackingClassifier(
    estimators=[
        ('nb', GaussianNB()),  # Modelo rápido e leve
        ('dt', DecisionTreeClassifier(max_depth=3)),  # Árvore de decisão rasa
    ],
    final_estimator=LogisticRegression(max_iter=100),  # Meta-modelo simples
    n_jobs=-1
)

# Dicionário de classificadores inicial:
classifiers = {
    'KNN': knn,             # Bayesian Search
    'SVM': svm,
    'Decision Tree': tree,  # Bayesian Search
    'LVQ': lvq,
    'MLP': mlp,
    'Ensemble Neural Network': ann_ensemble,
    'Stacking': stacking,
    'Random Forest': rf,    #Optuna
    'XGBoost': xgb,         #Optuna
    'LightGBM': lgbm
}

c:\Users\willi\Documents\Projetos\mestrado_mineracao\mda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Tunning Usando Bayesian e Optuna

In [ ]:
# Usando BS e Optuna em dois classificadores diferentes:

# Filtrar warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('sklearn').setLevel(logging.ERROR)

# Configurações para os modelos que costumam gerar warnings
import os
os.environ['PYTHONWARNINGS'] = 'ignore'

# Configurações específicas para XGBoost
from xgboost import set_config
set_config(verbosity=0)

# Espaços de busca para Bayesian Search: KNN e Decision Tree
param_spaces = {
    'KNN': {
        'n_neighbors': (1, 30),
        'weights': ['uniform', 'distance'],
        'p': (1, 2)
    },
    'Decision Tree': {
        'max_depth': (1, 50),
        'min_samples_split': (2, 20),
        'min_samples_leaf': (1, 20),
        'criterion': ['gini', 'entropy']
    }

}

# Avaliar classificadores e buscar hiperparâmetros com BayesSearchCV
results = {}
best_params = {}

for name, clf in classifiers.items():
    if name in param_spaces:
        print(f"Otimizando hiperparâmetros para {name}...")
        bayes_search = BayesSearchCV(
            estimator=clf,
            search_spaces=param_spaces[name],
            n_iter=50,
            cv=kfold,
            scoring='accuracy',
            n_jobs=-1,
            random_state=42
        )

        # Garante que X_train_resampled seja um DataFrame com os nomes de coluna corretos
        X_train_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)

        bayes_search.fit(X_train_resampled, y_train_resampled)
        best_params[name] = bayes_search.best_params_
        clf = bayes_search.best_estimator_
        print(f"Melhores parâmetros para {name}: {bayes_search.best_params_}\n")

#Optuna para RF e XGB

# Funções de objetivo para Optuna
def rf_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 200)
    max_depth = trial.suggest_int("max_depth", 2, 32, log=True)
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    return cross_val_score(clf, X_train_resampled, y_train_resampled, cv=kfold, scoring="accuracy").mean()

def xgb_objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True)
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    max_depth = trial.suggest_int("max_depth", 3, 10)

    # Encode the target variable to numeric values
    le = LabelEncoder()
    y_train_resampled_encoded = le.fit_transform(y_train_resampled)

    clf = XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators,
                        max_depth=max_depth, use_label_encoder=False, eval_metric='logloss')
    return cross_val_score(clf, X_train_resampled, y_train_resampled_encoded, cv=kfold, scoring="accuracy").mean()

# Otimização com Optuna
rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(rf_objective, n_trials=5)
classifiers['Random Forest'] = RandomForestClassifier(**rf_study.best_trial.params)
print("Melhores hiperparâmetros para Random Forest:", rf_study.best_trial.params)

xgb_study = optuna.create_study(direction="maximize")
xgb_study.optimize(xgb_objective, n_trials=5)
classifiers['XGBoost'] = XGBClassifier(**xgb_study.best_trial.params, use_label_encoder=False, eval_metric='logloss')
print("Melhores hiperparâmetros para XGBoost:", xgb_study.best_trial.params)

## Resumo Melhores Parâmetros

In [9]:
# Obter os parâmetros do classificador e remover os que têm valor None
xgb_clean_params = {k: v for k, v in classifiers['XGBoost'].get_params().items() if v is not None}

# Criar um novo classificador com os parâmetros limpos
cleaned_xgb = XGBClassifier(**xgb_clean_params)

print("Parâmetros do XGBoost sem valores None:")
print(xgb_clean_params)

Parâmetros do XGBoost sem valores None:
{'objective': 'binary:logistic', 'enable_categorical': False, 'eval_metric': 'logloss', 'missing': nan, 'use_label_encoder': False}


In [10]:
# Como explodir um dicionário:
# 'XGBoost': XGBClassifier(**xgb_study.best_trial.params),# Otimizado com Optuna

# Converter parâmetros otimizados para inicializar os classificadores
best_classifiers = {
    'KNN': KNeighborsClassifier(n_neighbors=1, p=1, weights='uniform'), # Otimizado com Bayesian Search
    'SVM': svm,  # Nenhuma otimização aplicada
    'Decision Tree': DecisionTreeClassifier(criterion='entropy', max_depth=36, min_samples_leaf=1, min_samples_split=2), # Otimizado com Bayesian Search
    'LVQ': lvq,  # Nenhuma otimização aplicada
    'MLP': mlp,  # Nenhuma otimização aplicada
    'Ensemble Neural Network': ann_ensemble,  # Nenhuma otimização aplicada
    'Stacking': stacking,  # Nenhuma otimização aplicada
    'Random Forest': RandomForestClassifier(max_depth=28, n_estimators=97), # Otimizado com Optuna    
    'XGBoost': XGBClassifier(objective='binary:logistic', enable_categorical=False, eval_metric='logloss', 
                             learning_rate=0.09504317284612004, max_depth=6, n_estimators=188),  # Otimizado com Optuna
    'LightGBM': lgbm # Nenhuma otimização aplicada
}

# Mostrar os melhores parâmetros para os classificadores otimizados
print("Classificadores com os melhores parâmetros:")

for name, model in best_classifiers.items():
  print(f"{name}: {model}")

Classificadores com os melhores parâmetros:
KNN: KNeighborsClassifier(n_neighbors=1, p=1)
SVM: SVC(probability=True)
Decision Tree: DecisionTreeClassifier(criterion='entropy', max_depth=36)
LVQ: MLPClassifier(learning_rate_init=0.1, max_iter=100, solver='sgd')
MLP: MLPClassifier(hidden_layer_sizes=(50,), max_iter=100)
Ensemble Neural Network: VotingClassifier(estimators=[('mlp_relu',
                              MLPClassifier(hidden_layer_sizes=(50, 20),
                                            max_iter=100)),
                             ('mlp_tanh',
                              MLPClassifier(activation='tanh',
                                            hidden_layer_sizes=(50, 20),
                                            max_iter=100)),
                             ('mlp_logistic',
                              MLPClassifier(activation='logistic',
                                            hidden_layer_sizes=(50, 20),
                                            max_iter=100

## Avaliando os modelos após Tunning

In [11]:
import warnings
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_validate

# Filtrar warnings
warnings.filterwarnings('ignore')

def calculate_acsa(y_true, y_pred):
    classes = np.unique(y_true)
    class_accuracies = [
        accuracy_score(y_true[y_true == c], y_pred[y_true == c]) for c in classes
    ]
    return np.mean(class_accuracies)

def evaluate_model(model, X, y, kfold):
    # Converter para array numpy se for DataFrame/Series
    X_numpy = X.values if hasattr(X, 'values') else X
    y_numpy = y.values if hasattr(y, 'values') else y
    
    # Usar cross_validate para calcular accuracy, f1 e recall em uma única chamada
    scoring = ['accuracy', 'f1', 'recall']
    scores = cross_validate(model, X_numpy, y_numpy, cv=kfold, scoring=scoring, return_estimator=True)

    # Métricas médias da validação cruzada
    accuracy = scores['test_accuracy'].mean()
    f1 = scores['test_f1'].mean()
    recall = scores['test_recall'].mean()

    # Cálculo do ACSA manualmente
    acsa_scores = []
    for estimator, (train_idx, val_idx) in zip(scores['estimator'], kfold.split(X_numpy, y_numpy)):
        X_val_fold = X_numpy[val_idx]
        y_val_fold = y_numpy[val_idx]
        y_pred_fold = estimator.predict(X_val_fold)
        acsa_scores.append(calculate_acsa(y_val_fold, y_pred_fold))
    acsa = np.mean(acsa_scores)

    # Ajustar modelo nos dados completos e calcular a matriz de confusão
    model.fit(X_numpy, y_numpy)
    y_pred = model.predict(X_numpy)
    conf_matrix = confusion_matrix(y_numpy, y_pred)

    return accuracy, f1, recall, acsa, conf_matrix

# Uso do código
results = {}
for name, clf in classifiers.items():
    # Avaliar desempenho do classificador
    accuracy, f1, recall, acsa, conf_matrix = evaluate_model(clf, X_train_resampled, y_train_resampled, kfold)
    
    results[name] = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Recall': recall,
        'ACSA': acsa,
        'Confusion Matrix': conf_matrix
    }
    
    print(f"{name} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Recall: {recall:.4f}, ACSA: {acsa:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}\n")

# Resultados
print("\nResumo dos melhores parâmetros:")
for name, params in best_params.items():
    print(f"{name}: {params}")

KNN - Accuracy: 0.9586, F1 Score: 0.9572, Recall: 0.9179, ACSA: 0.9590
Confusion Matrix:
[[7821    0]
 [ 479 7488]]

SVM - Accuracy: 0.9680, F1 Score: 0.9673, Recall: 0.9386, ACSA: 0.9683
Confusion Matrix:
[[7814    7]
 [ 461 7506]]

Decision Tree - Accuracy: 0.9839, F1 Score: 0.9840, Recall: 0.9812, ACSA: 0.9839
Confusion Matrix:
[[7821    0]
 [   0 7967]]

LVQ - Accuracy: 0.9936, F1 Score: 0.9936, Recall: 0.9903, ACSA: 0.9936
Confusion Matrix:
[[7807   14]
 [   3 7964]]

MLP - Accuracy: 0.9895, F1 Score: 0.9895, Recall: 0.9828, ACSA: 0.9895
Confusion Matrix:
[[7811   10]
 [  45 7922]]

Ensemble Neural Network - Accuracy: 0.9949, F1 Score: 0.9950, Recall: 0.9915, ACSA: 0.9950
Confusion Matrix:
[[7821    0]
 [  25 7942]]

Stacking - Accuracy: 0.8867, F1 Score: 0.8892, Recall: 0.9030, ACSA: 0.8865
Confusion Matrix:
[[6682 1139]
 [ 598 7369]]

Random Forest - Accuracy: 0.9912, F1 Score: 0.9912, Recall: 0.9864, ACSA: 0.9912
Confusion Matrix:
[[7821    0]
 [   0 7967]]

XGBoost - Accuracy:

## Amostra para análise Estatística

In [12]:
import warnings
import logging
import os
import numpy as np
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
from xgboost import set_config

# Filtrar warnings
warnings.filterwarnings('ignore')
logging.getLogger('sklearn').setLevel(logging.ERROR)
os.environ['PYTHONWARNINGS'] = 'ignore'
set_config(verbosity=0)

def calculate_acsa(y_true, y_pred):
    """Calcula o Average Class-Specific Accuracy (ACSA)."""
    classes = np.unique(y_true)
    class_accuracies = [
        accuracy_score(y_true[y_true == c], y_pred[y_true == c]) for c in classes
    ]
    return np.mean(class_accuracies)

def evaluate_model(model, X, y, kfold, n_samples=10):
    """
    Avalia o modelo usando validação cruzada múltiplas vezes.
    
    Args:
        model: Modelo de classificação
        X: Features (DataFrame ou array)
        y: Target (Series ou array)
        kfold: Objeto de validação cruzada
        n_samples: Número de iterações de avaliação
    """
    # Converter para numpy arrays se necessário
    X_numpy = X.values if hasattr(X, 'values') else np.array(X)
    y_numpy = y.values if hasattr(y, 'values') else np.array(y)
    
    results = {
        'Accuracy': [],
        'F1 Score': [],
        'Recall': [],
        'ACSA': [],
        'Confusion Matrix': []
    }
    
    for sample in range(n_samples):
        try:
            # Realizando a validação cruzada
            scores = cross_validate(
                model, 
                X_numpy, 
                y_numpy, 
                cv=kfold, 
                scoring=['accuracy', 'f1', 'recall'],
                return_estimator=True,
                n_jobs=-1  # Usar todos os cores disponíveis
            )

            # Métricas médias da validação cruzada
            results['Accuracy'].append(scores['test_accuracy'].mean())
            results['F1 Score'].append(scores['test_f1'].mean())
            results['Recall'].append(scores['test_recall'].mean())

            # Cálculo do ACSA
            acsa_scores = []
            for estimator, (_, val_idx) in zip(scores['estimator'], kfold.split(X_numpy, y_numpy)):
                X_val = X_numpy[val_idx]
                y_val = y_numpy[val_idx]
                y_pred = estimator.predict(X_val)
                acsa_scores.append(calculate_acsa(y_val, y_pred))
            
            results['ACSA'].append(np.mean(acsa_scores))

            # Matriz de confusão usando o último modelo ajustado
            model.fit(X_numpy, y_numpy)
            y_pred_final = model.predict(X_numpy)
            results['Confusion Matrix'].append(confusion_matrix(y_numpy, y_pred_final))

        except Exception as e:
            print(f"Erro na amostra {sample}: {str(e)}")
            continue
            
    return results

def print_results(results_dict):
    """Imprime os resultados de forma organizada."""
    for name, metrics in results_dict.items():
        print(f"\n{'='*50}")
        print(f"Resultados para {name}:")
        print(f"{'='*50}")
        
        # Métricas numéricas
        for metric in ['Accuracy', 'F1 Score', 'Recall', 'ACSA']:
            values = metrics[metric]
            if values:  # Verifica se há valores
                mean = np.mean(values)
                std = np.std(values)
                print(f"\n{metric}:")
                print(f"  Média: {mean:.4f}")
                print(f"  Desvio Padrão: {std:.4f}")
                print(f"  95% IC: [{mean - 1.96*std/np.sqrt(len(values)):.4f}, "
                      f"{mean + 1.96*std/np.sqrt(len(values)):.4f}]")
        
        # Matriz de Confusão
        print("\nMatriz de Confusão (média):")
        if metrics['Confusion Matrix']:
            mean_conf_matrix = np.mean(metrics['Confusion Matrix'], axis=0)
            print(mean_conf_matrix.astype(int))

# Uso do código
if __name__ == "__main__":
    # Definindo a validação cruzada
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    # Inicializando resultados
    results = {}
    
    # Avaliando cada classificador
    for name, clf in classifiers.items():
        print(f"\nAvaliando {name}...")
        results[name] = evaluate_model(
            clf, 
            X_train_resampled, 
            y_train_resampled, 
            kfold, 
            n_samples=10
        )
    
    # Imprimindo resultados
    print_results(results)


Avaliando KNN...

Avaliando SVM...


KeyboardInterrupt: 

# Roteiro

- Registro dos resultados em **dataframe**
- Visualização de Resultados: Gráficos de caixa (boxplot), barras, etc.
- Análise Estatística: Relatório de classificação e testes de significância estatística.
- Aplicar o protocolo utilizado de comparação de classificadores de Janez Demsar.
- Apresentar desempenhos de treinamento e teste para todos os modelos.
- Análise de Custo-Benefício: Avaliar os recursos computacionais utilizados por cada modelo (tempo de processamento e memória) em relação ao desempenho alcançado.
- Teste de Estresse dos Modelos: Realizar testes com dados novos e desconhecidos para avaliar a robustez dos modelos.
- Métricas de Complexidade do Modelo: Avaliar a complexidade dos modelos, como número de parâmetros e tempo de inferência.
- Explainable AI (XAI): Explicar as previsões de pelo menos dois dos modelos utilizando ferramentas de XAI, como SHAP ou LIME, para aumentar a compreensão sobre os fatores que influenciam as decisões dos modelos. Escolher o melhor modelo a partir da comparação estatística realizada para aplicar o XAI.